# Korean BERT pre-trained cased (KoBERT) for Huggingface Transformers
모델 다운로드 서비스가 작동을 중지 -> 모델 다운로드 방식을 Hugging Face를 통한 모델 다운로드로 전환
https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf/kobert_tokenizer

참고 블로그
1. https://complexoftaste.tistory.com/2

2. https://velog.io/@j_aion/iOS-KoBERT-%EA%B0%90%EC%84%B1-%EB%B6%84%EC%84%9D-%EB%AA%A8%EB%8D%B8%EB%A7%81
3. https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab
4. https://sig413.tistory.com/m/80
5. test_acc가 낮게 나오는 문제에 대해서 : https://hoit1302.tistory.com/159
6. https://github.com/ChangZero/koBERT-finetuning-demo/blob/main/kobert_colab.ipynb




## 오류 참고 링크
1. https://github.com/SKTBrain/KoBERT/issues/104
2. https://blog.naver.com/newyearchive/223097878715



## 환경설정

In [3]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch

In [4]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-duzdqoqn/kobert-tokenizer_3386b7dce5384227ad1a852fa8acf09e
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-duzdqoqn/kobert-tokenizer_3386b7dce5384227ad1a852fa8acf09e
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=c509ccf4a0427f33222db208695af0106567663a4aab7c42b97c25ef0584dc88
  Stored in directory: /tmp/pip-ephem-wheel-cache-1bnzll_7/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [5]:
# 구글 코랩에서 파이토치, ML 모델링 임포트
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from sklearn.model_selection import train_test_split





# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup




#드라이브 연동
from google.colab import drive
drive.mount('/content/drive')


#GPU 연결 해야함 -> 안 하면 오류 발생
device = torch.device("cuda:0")

# # Torch GPU 설정
# device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = torch.device(device_type)


/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


Mounted at /content/drive


In [6]:
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [7]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [8]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [9]:
OneOff_data = pd.read_excel("/content/drive/MyDrive/Prog_All/Data/ko_단발성_대화_데이터셋.xlsx", engine='openpyxl')
continuity_data = pd.read_excel("/content/drive/MyDrive/Prog_All/Data/ko_연속적_대화_데이터셋.xlsx", engine='openpyxl')

In [10]:
OneOff_data.head(10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [11]:
type(OneOff_data)

pandas.core.frame.DataFrame

In [12]:
continuity_data.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,행복,중립,슬픔,공포,혐오,분노,놀람
0,dialog #,발화,감정,NaN,NaN,1030.0,NaN,NaN,NaN,NaN,NaN,NaN
1,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,손님 왔어요.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,손님? 누구?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
55624,S,얘긴 다 끝났냐? 원예부,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55625,NaN,"예. 그거 때문에, 부탁이 있......는......데요.",중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55626,NaN,여자 숨겨달라는거면 사절이다.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55627,NaN,아무래도 안되나요?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
OneOff_data = OneOff_data[['Sentence','Emotion']]
continuity_data = continuity_data[['Unnamed: 1','Unnamed: 2']]
continuity_data.drop([0],axis=0,inplace=True)
continuity_data.rename(columns={'Unnamed: 1':'Sentence','Unnamed: 2':'Emotion'},inplace=True)
continuity_data.replace('ㅍ','공포',inplace=True)
continuity_data.replace(['분','분ㄴ'],'분노',inplace=True)
continuity_data.replace(['ㅈ중립','중림','ㄴ중립','줄'],'분노',inplace=True)

<ipython-input-13-8cd63477b64b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.drop([0],axis=0,inplace=True)
<ipython-input-13-8cd63477b64b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.rename(columns={'Unnamed: 1':'Sentence','Unnamed: 2':'Emotion'},inplace=True)
<ipython-input-13-8cd63477b64b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.replace('ㅍ','공포',inplace=True)
<ipython-input-13-8cd63477b64b>:6: Sett

In [14]:
print(type(OneOff_data),type(continuity_data))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [ ]:
# OneOff_data = OneOff_data.rename(columns={"발화":'sentence','감정':'str','감정.1':'Emotion'})
# continuity_data= continuity_data.rename(columns={"발화":'sentence','감정_str':'str','감정_int':'Emotion'})

In [ ]:
# data = pd.concat([OneOff_data, continuity_data], axis=0)
# data = data.replace({'Emotion' : 7}, 0)

In [15]:
data = pd.concat([OneOff_data,continuity_data])
data.sample(n=10)

In [16]:
type(data)

pandas.core.frame.DataFrame

In [17]:
data.drop_duplicates(['Sentence','Emotion'],inplace=True) # 중복행 제거
len(data)

90123

In [18]:
data

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
55624,얘긴 다 끝났냐? 원예부,중립
55625,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,여자 숨겨달라는거면 사절이다.,중립
55627,아무래도 안되나요?,중립


In [19]:
data.loc[(data['Emotion'] == "공포"), 'label'] = 0  # 공포 → 0 fear
data.loc[(data['Emotion'] == "놀람"), 'label'] = 1  # 놀람 → 1 surprise
data.loc[(data['Emotion'] == "분노"), 'label'] = 2  # 분노 → 2 anger
data.loc[(data['Emotion'] == "슬픔"), 'label'] = 3  # 슬픔 → 3 sadness
data.loc[(data['Emotion'] == "중립"), 'label'] = 4  # 중립 → 4 neutral
data.loc[(data['Emotion'] == "행복"), 'label'] = 5  # 행복 → 5 happiness
data.loc[(data['Emotion'] == "혐오"), 'label'] = 6  # 혐오 → 6 disgust

In [ ]:
data_list = []
for q, label in zip(data['Sentence'], data['Emotion'])  :
    check_data = []
    check_data.append(q)
    check_data.append(str(label))

    data_list.append(check_data)
data_list

In [21]:
data['Emotion'].value_counts()

중립    45641
놀람     9866
분노     9238
슬픔     7167
행복     7015
혐오     5621
공포     5568
Name: Emotion, dtype: int64

In [22]:
fear = data[data['Emotion'] == "공포"]
surprise = data[data['Emotion'] == "놀람"]
anger = data[data['Emotion'] == "분노"]
sadness = data[data['Emotion'] == "슬픔"]
neutral = data[data['Emotion'] == "중립"]
happiness = data[data['Emotion'] == "행복"]
disgust = data[data['Emotion'] == "혐오"]

In [23]:
fear_train = fear[:int(len(fear)*0.8)]
surprise_train = surprise[:int(len(surprise)*0.8)]
anger_train = anger[:int(len(anger)*0.8)]
sadness_train = sadness[:int(len(sadness)*0.8)]
neutral_train = neutral[:int(len(neutral)*0.8)]
happiness_train = happiness[:int(len(happiness)*0.8)]
disgust_train = disgust[:int(len(disgust)*0.8)]

In [24]:
fear_test = fear[int(len(fear)*0.8):]
surprise_test = surprise[int(len(surprise)*0.8):]
anger_test = anger[int(len(anger)*0.8):]
sadness_test = sadness[int(len(sadness)*0.8):]
neutral_test = neutral[int(len(neutral)*0.8):]
happiness_test = happiness[int(len(happiness)*0.8):]
disgust_test = disgust[int(len(disgust)*0.8):]

In [25]:
train = pd.concat([fear_train, surprise_train, anger_train, sadness_train, neutral_train, happiness_train, disgust_train],ignore_index=True)
test = pd.concat([fear_test, surprise_test, anger_test, sadness_test, neutral_test, happiness_test, disgust_test],ignore_index=True)

In [26]:
train.shape

(72089, 3)

In [27]:
train_data = []
for ques, label in zip(train['Sentence'], train['label'])  :
    data = []
    data.append(ques)
    data.append(str(int(label)))

    train_data.append(data)

In [28]:
test_data = []
for ques, label in zip(test['Sentence'], test['label'])  :
    data = []
    data.append(ques)
    data.append(str(int(label)))

    test_data.append(data)

In [29]:
train_df = pd.DataFrame(train_data)

In [30]:
# 중립이 많음.
train_df[1].value_counts()

4    36512
1     7892
2     7390
3     5733
5     5612
6     4496
0     4454
Name: 1, dtype: int64

## 모델불러오기
참조 블로그: https://github.com/ChangZero/koBERT-finetuning-demo/blob/main/kobert_colab.ipynb

In [31]:

class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')





## Model Class & Funcs

In [32]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [33]:
data_train = BERTDataset(train_data, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(test_data, 0, 1, tokenizer, vocab, max_len, True, False)

In [34]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [35]:
### KoBERT 학습모델

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 7개로 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [37]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [38]:


# Prepare optimizer and schedule (linear warmup and decay)
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [39]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

In [40]:
# KoBERT 모델 학습시키기

train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-40-8ac5b9e821f6>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1127 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.350369930267334 train acc 0.03125
epoch 1 batch id 201 loss 4.959075927734375 train acc 0.6526741293532339
epoch 1 batch id 401 loss 7.5373053550720215 train acc 0.7722100997506235
epoch 1 batch id 601 loss 0.0010787262581288815 train acc 0.8317127703826955
epoch 1 batch id 801 loss 0.0004307316557969898 train acc 0.8737320536828964
epoch 1 batch id 1001 loss 0.010143266059458256 train acc 0.8861763236763237
epoch 1 train acc 0.8864934560780834


<ipython-input-40-8ac5b9e821f6>:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/282 [00:00<?, ?it/s]

...

epoch 4 test acc 0.06349734042553191


  0%|          | 0/1127 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 4.652949810028076 train acc 0.0
epoch 5 batch id 201 loss 5.753662109375 train acc 0.8123445273631841
epoch 5 batch id 401 loss 2.9106101989746094 train acc 0.7697163341645885
epoch 5 batch id 601 loss 0.004722312092781067 train acc 0.8359504991680532
epoch 5 batch id 801 loss 0.003359145950525999 train acc 0.876911672908864
epoch 5 batch id 1001 loss 6.218998432159424 train acc 0.8689903846153846
epoch 5 train acc 0.7718361801242236


  0%|          | 0/282 [00:00<?, ?it/s]

epoch 5 test acc 0.5058178191489362


## predict :
주어진 문장이 현재 학습이 완료된 모델 내에서 어떤 라벨과 argmax인지 판단하고 추론된 결과를 리턴하는 함수

In [41]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        print(out)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            print(logits)
            print(np.argmax(logits))

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [45]:
# #version2
# # 토큰화
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# def predict(predict_sentence):

#     data = [predict_sentence, '0']
#     dataset_another = [data]

#     another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
#     test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

#     model.eval()

#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)

#         valid_length= valid_length
#         label = label.long().to(device)

#         out = model(token_ids, valid_length, segment_ids)


# #         test_eval=[]
#         for i in out:
#             logits=i
#             logits = logits.detach().cpu().numpy()
#             emotion = emotion_dict[np.argmax(logits)]


#         print(f">> 입력하신 내용의 감정은 {emotion}입니다.")


In [42]:
predict_sentence = '영화에 나오는 귀신이 너무 무섭네요'
predict(predict_sentence)

tensor([[-4.8178, -1.7435, -3.7298,  1.2121,  8.6504,  2.7403, -3.5895]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
[-4.8178    -1.7434648 -3.7297664  1.2121292  8.650417   2.7402937
 -3.589507 ]
4
>> 입력하신 내용에서 중립이 느껴집니다.


In [43]:
#질문 무한반복하기! 0 입력시 종료
while True:
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        print("감정 분석을 종료합니다.")
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 나 너무 슬퍼
tensor([[-4.8171, -1.7424, -3.7274,  1.2163,  8.6511,  2.7353, -3.5929]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
[-4.8171253 -1.7423987 -3.72742    1.2162739  8.651139   2.7352588
 -3.5928893]
4
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 나 무서워
tensor([[-4.8165, -1.7414, -3.7253,  1.2201,  8.6518,  2.7306, -3.5960]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
[-4.816515  -1.7414415 -3.7252612  1.2200959  8.651793   2.7306416
 -3.5959787]
4
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 0
